In [5]:
import pandas as pd
import csv
import os
import boto3
from io import StringIO

# Load the raw data from S3
def load_data_from_s3(bucket_name, file_key):
    s3_client = boto3.client('s3')
    try:
        response = s3_client.get_object(Bucket=bucket_name, Key=file_key)
        content = response['Body'].read().decode('utf-8')
        data = pd.read_csv(StringIO(content), sep=',')  # Using comma as separator for Patent_Data.csv
        # Clean column names (remove any trailing \r)
        data.columns = [col.strip('\r') for col in data.columns]
        return data
    except Exception as e:
        print(f"Error loading data from S3: {e}")
        raise

# Create node CSV files for Neptune
def create_node_files(data, output_dir):
    # Ensure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # 1. Case Nodes
    case_nodes = data[['case_no', 'filed_date', 'filing_year', 'status', 'court', 
                        'patents', 'cause_of_action', 'closed_date']].copy()
    case_nodes['~id'] = case_nodes['case_no']
    case_nodes['~label'] = 'Case'
    case_nodes.to_csv(f"{output_dir}/case_nodes.csv", index=False)
    
    # 2. Company Nodes (from both plaintiffs and defendants)
    # Extract all plaintiff companies
    plaintiff_companies = set()
    for plaintiff_str in data['plaintiff'].dropna():
        for plaintiff in plaintiff_str.split('|'):
            plaintiff_companies.add(plaintiff.strip())
            
    # Extract all defendant companies
    defendant_companies = set()
    for defendant_str in data['defendant'].dropna():
        for defendant in defendant_str.split('|'):
            defendant_companies.add(defendant.strip())
            
    # Combine all companies and create nodes
    all_companies = list(plaintiff_companies.union(defendant_companies))
    company_nodes = pd.DataFrame({
        '~id': all_companies,
        '~label': 'Company'
    })
    company_nodes.to_csv(f"{output_dir}/company_nodes.csv", index=False)
    
    # 3. Judge Nodes
    judge_nodes = pd.DataFrame({
        '~id': data['judge'].dropna().unique(),
        '~label': 'Judge'
    })
    judge_nodes.to_csv(f"{output_dir}/judge_nodes.csv", index=False)
    
    # 4. Court Nodes
    court_nodes = pd.DataFrame({
        '~id': data['court'].dropna().unique(),
        '~label': 'Court'
    })
    court_nodes.to_csv(f"{output_dir}/court_nodes.csv", index=False)
    
    # 5. Patent Nodes (if patent numbers are available)
    patent_ids = set()
    for patent_str in data['patents'].dropna():
        for patent in patent_str.split('|'):
            if patent.strip():
                patent_ids.add(patent.strip())
                
    if patent_ids:
        patent_nodes = pd.DataFrame({
            '~id': list(patent_ids),
            '~label': 'Patent'
        })
        patent_nodes.to_csv(f"{output_dir}/patent_nodes.csv", index=False)

# Create edge CSV files for Neptune
def create_edge_files(data, output_dir):
    # Ensure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # 1. FILED_LAWSUIT edges (plaintiff -> case)
    filed_lawsuit_rows = []
    for _, row in data.iterrows():
        case_id = row['case_no']
        if pd.notna(row['plaintiff']):
            for plaintiff in row['plaintiff'].split('|'):
                plaintiff = plaintiff.strip()
                filed_lawsuit_rows.append({
                    '~from': plaintiff,
                    '~to': case_id,
                    '~label': 'FILED_LAWSUIT',
                    'filed_date': row['filed_date']
                })
    
    filed_lawsuit_df = pd.DataFrame(filed_lawsuit_rows)
    if not filed_lawsuit_df.empty:
        filed_lawsuit_df.to_csv(f"{output_dir}/filed_lawsuit_edges.csv", index=False)
    
    # 2. DEFENDANT_IN edges (defendant -> case)
    defendant_rows = []
    for _, row in data.iterrows():
        case_id = row['case_no']
        if pd.notna(row['defendant']):
            for defendant in row['defendant'].split('|'):
                defendant = defendant.strip()
                defendant_rows.append({
                    '~from': defendant,
                    '~to': case_id,
                    '~label': 'DEFENDANT_IN'
                })
    
    defendant_df = pd.DataFrame(defendant_rows)
    if not defendant_df.empty:
        defendant_df.to_csv(f"{output_dir}/defendant_edges.csv", index=False)
    
    # 3. PRESIDED_OVER edges (judge -> case)
    judge_rows = []
    for _, row in data.iterrows():
        case_id = row['case_no']
        if pd.notna(row['judge']):
            judge_rows.append({
                '~from': row['judge'],
                '~to': case_id,
                '~label': 'PRESIDED_OVER'
            })
    
    judge_df = pd.DataFrame(judge_rows)
    if not judge_df.empty:
        judge_df.to_csv(f"{output_dir}/judge_edges.csv", index=False)
    
    # 4. VENUE_FOR edges (court -> case)
    court_rows = []
    for _, row in data.iterrows():
        case_id = row['case_no']
        if pd.notna(row['court']):
            court_rows.append({
                '~from': row['court'],
                '~to': case_id,
                '~label': 'VENUE_FOR'
            })
    
    court_df = pd.DataFrame(court_rows)
    if not court_df.empty:
        court_df.to_csv(f"{output_dir}/court_edges.csv", index=False)
    
    # 5. INVOLVES_PATENT edges (case -> patent)
    patent_rows = []
    for _, row in data.iterrows():
        case_id = row['case_no']
        if pd.notna(row['patents']):
            for patent in row['patents'].split('|'):
                patent = patent.strip()
                if patent:
                    patent_rows.append({
                        '~from': case_id,
                        '~to': patent,
                        '~label': 'INVOLVES_PATENT'
                    })
    
    patent_df = pd.DataFrame(patent_rows)
    if not patent_df.empty:
        patent_df.to_csv(f"{output_dir}/patent_edges.csv", index=False)

# Main function to process the data from S3
def process_patent_data_for_neptune_from_s3(s3_bucket, s3_key, output_dir, output_s3_bucket=None):
    print(f"Loading data from s3://{s3_bucket}/{s3_key}...")
    data = load_data_from_s3(s3_bucket, s3_key)
    
    print(f"Creating node files in {output_dir}...")
    create_node_files(data, output_dir)
    
    print(f"Creating edge files in {output_dir}...")
    create_edge_files(data, output_dir)
    
    # Optionally upload results back to S3
    if output_s3_bucket:
        upload_results_to_s3(output_dir, output_s3_bucket)
        print(f"Uploaded results to s3://{output_s3_bucket}/neptune_import/")
    else:
        print(f"Done! Files are ready for Neptune import in {output_dir}")

# Function to upload results back to S3
def upload_results_to_s3(local_dir, s3_bucket, s3_prefix='neptune_import'):
    s3_client = boto3.client('s3')
    for filename in os.listdir(local_dir):
        local_path = os.path.join(local_dir, filename)
        s3_key = f"{s3_prefix}/{filename}"
        print(f"Uploading {local_path} to s3://{s3_bucket}/{s3_key}")
        s3_client.upload_file(local_path, s3_bucket, s3_key)

# Example usage
if __name__ == "__main__":
    # Replace these with your actual S3 bucket and key
    S3_BUCKET = "team4bucket-mmm"  # <-- INSERT YOUR BUCKET NAME HERE
    S3_KEY = "Patent_Data_Kaggle/Patent_Data.csv"    # <-- Path to your Patent_Data.csv file
    
    # Local directory for output files
    OUTPUT_DIR = "relationships"
    
    # Optional: S3 bucket for output files (if you want to upload results back to S3)
    OUTPUT_S3_BUCKET = "team4bucket-mmm"  # <-- Optional, remove if not needed
    
    process_patent_data_for_neptune_from_s3(S3_BUCKET, S3_KEY, OUTPUT_DIR, OUTPUT_S3_BUCKET)

Loading data from s3://team4bucket-mmm/Patent_Data_Kaggle/Patent_Data.csv...


/tmp/ipykernel_7910/992024916.py:13: DtypeWarning: Columns (13,14,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(StringIO(content), sep=',')  # Using comma as separator for Patent_Data.csv


Creating node files in relationships...
Creating edge files in relationships...
Uploading relationships/case_nodes.csv to s3://team4bucket-mmm/neptune_import/case_nodes.csv
Uploading relationships/company_nodes.csv to s3://team4bucket-mmm/neptune_import/company_nodes.csv
Uploading relationships/judge_nodes.csv to s3://team4bucket-mmm/neptune_import/judge_nodes.csv
Uploading relationships/court_nodes.csv to s3://team4bucket-mmm/neptune_import/court_nodes.csv
Uploading relationships/patent_nodes.csv to s3://team4bucket-mmm/neptune_import/patent_nodes.csv
Uploading relationships/filed_lawsuit_edges.csv to s3://team4bucket-mmm/neptune_import/filed_lawsuit_edges.csv
Uploading relationships/defendant_edges.csv to s3://team4bucket-mmm/neptune_import/defendant_edges.csv
Uploading relationships/judge_edges.csv to s3://team4bucket-mmm/neptune_import/judge_edges.csv
Uploading relationships/court_edges.csv to s3://team4bucket-mmm/neptune_import/court_edges.csv
Uploading relationships/patent_edges.